In [ ]:
import json
import os
import sys

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gc_utils import iteration_name, snapshot_name  # type: ignore
from scipy.interpolate import PchipInterpolator, interp1d

Lets first make a plot that shows the mass of Ex-situ formed GCs

In [52]:
def get_type_flag(gc, grp, gc_survive_lst):
    if gc in gc_survive_lst:
        if grp == 0:
            # formed in-situ and survived to z = 0
            type_flag = 0
        elif grp > 0:
            # formed ex-situ, accreted and survived to z = 0
            type_flag = 2

    elif grp < -2:
        # formed ex-situ and died before accretion
        type_flag = 4

    elif grp > 0:
        # formed ex-situ, accreted but died before z = 0
        type_flag = 3

    elif grp == 0:
        # formed in-situ, but died before z = 0
        type_flag = 1

    else:
        sys.exit("Some GC Missing Type Flag")

    return type_flag

In [71]:
def get_mass_data(it, evolve_mass_loss, sim, sim_dir, save_dir, save_dict=True, return_dict=False):
    # get required files

    proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
    proc_data = h5py.File(proc_file, "r")  # open processed data file

    pub_data = sim_dir + "snapshot_times_public.txt"
    pub_snaps = pd.read_table(pub_data, comment="#", header=None, sep=r"\s+")
    pub_snaps.columns = [
        "index",
        "scale_factor",
        "redshift",
        "time_Gyr",
        "lookback_time_Gyr",
        "time_width_Myr",
    ]

    mass_dict = {}

    it_id = iteration_name(it)
    mass_dict[it_id] = {}

    src_dat = proc_data[it_id]["source"]
    ana_mask = np.array(src_dat["analyse_flag"]) == 1

    gc_lst = np.array(src_dat["gc_id"])[ana_mask]
    gc_survive_lst = np.array(proc_data[it_id]["snapshots"]["snap600"]["gc_id"])

    for gc in gc_lst:
        gc_id = str(gc)
        mass_dict[it_id][gc_id] = {}

        idx = np.where(np.array(src_dat["gc_id"])[ana_mask] == gc)[0][0]
        grp = np.array(src_dat["group_id"])[ana_mask][idx]

        mass_dict[it_id][gc_id]["group_id"] = int(grp)
        mass_dict[it_id][gc_id]["type_flag"] = get_type_flag(gc, grp, gc_survive_lst)

        time_lst = []
        time_for_lst = []

        log_mass_lst = []  # mass at each time
        mass_loss_lst = []  # mass loss at each time[1:]
        mass_loss_det_lst = []  # mass loss at each time[1:] but taking t_form as 55% of t_form

        # get formation information
        t_form = np.array(src_dat["form_time"])[ana_mask][idx]

        log_mass_form = np.array(src_dat["logm_tform"])[ana_mask][idx]
        log_mass_form_det = (
            np.log10(1 - evolve_mass_loss) + log_mass_form
        )  # corrected for mass loss by evolution

        # get other details
        t_dis = np.array(src_dat["t_dis"])[ana_mask][idx]

        # update relevant lists
        time_lst.append(t_form)
        time_for_lst.append(t_form - t_form)

        # only for accreted (types 2, 3, 4) GCs
        if mass_dict[it_id][gc_id]["type_flag"] > 1:
            time_acc_lst = []

            t_acc = np.array(src_dat["t_acc"])[ana_mask][idx]
            time_acc_lst.append(t_form - t_acc)

        log_mass_lst.append(log_mass_form)

        if t_dis == -1:
            snap_lst = pub_snaps[(pub_snaps["time_Gyr"] >= t_form)]["index"]
        else:
            snap_lst = pub_snaps[(pub_snaps["time_Gyr"] >= t_form) & (pub_snaps["time_Gyr"] <= t_dis)][
                "index"
            ]

        # this has been added to ensure that GCs that form and die between snaps are considered in the for loop
        # that gets the details of their death (death loop)
        time = t_form

        for snap in snap_lst:
            snap_id = snapshot_name(snap)
            snap_dat = proc_data[it_id]["snapshots"][snap_id]

            # check, sometimes timing issues with GC formation model
            gc_snap_lst = np.array(snap_dat["gc_id"])
            if gc not in gc_snap_lst:
                continue

            time = pub_snaps[pub_snaps["index"] == snap]["time_Gyr"].values[0]

            snap_idx = np.where(np.array(snap_dat["gc_id"]) == gc)[0][0]
            cur_log_mass = np.array(snap_dat["mass"])[snap_idx]

            pst_log_mass = log_mass_lst[-1]
            pst_mass = 10**pst_log_mass

            cur_mass = 10**cur_log_mass

            # get mass loss (not logged)
            mass_los = pst_mass - cur_mass

            # make initial correction to detectable mass loss list
            if len(log_mass_lst) == 1:
                mass_form_det = 10**log_mass_form_det
                mass_los_det = mass_form_det - cur_mass
            else:
                mass_los_det = mass_los

            # update relevant lists
            log_mass_lst.append(cur_log_mass)

            mass_loss_lst.append(mass_los)
            mass_loss_det_lst.append(mass_los_det)

            time_lst.append(time)
            time_for_lst.append(time - t_form)

            # get accretion time
            if mass_dict[it_id][gc_id]["type_flag"] > 1:
                time_acc_lst.append(time - t_acc)

        # need to consider full disruption stuff too
        # like if in past list but not in current list mass loss is 100%

        # (death loop)
        # update full disruption details fro GCs that don't survive to z = 0

        # if (mass_dict[it_id][gc]["type_flag"] != 0) and (mass_dict[it_id][gc]["type_flag"] != 2):
        #     last_time = pub_snaps[pub_snaps["time_Gyr"] > time]["time_Gyr"].values[0]

        if t_dis != -1:
            # no longer exists
            # final_log_mass = np.nan
            final_log_mass = None

            # past mass
            pst_log_mass = log_mass_lst[-1]
            pst_mass = 10**pst_log_mass

            # as fully disrupted
            mass_los = pst_mass

            time_lst.append(t_dis)
            time_for_lst.append(t_dis - t_form)

            # get accretion time
            if mass_dict[it_id][gc_id]["type_flag"] > 1:
                time_acc_lst.append(t_dis - t_acc)

            log_mass_lst.append(final_log_mass)
            mass_loss_lst.append(mass_los)
            mass_loss_det_lst.append(mass_los)

        # update dictionary
        mass_dict[it_id][gc_id]["time"] = time_lst
        mass_dict[it_id][gc_id]["form_time"] = time_for_lst

        if mass_dict[it_id][gc_id]["type_flag"] > 1:
            mass_dict[it_id][gc_id]["acc_time"] = time_acc_lst

        mass_dict[it_id][gc_id]["log_mass"] = log_mass_lst
        mass_dict[it_id][gc_id]["mass_loss"] = mass_loss_lst
        mass_dict[it_id][gc_id]["mass_loss_detectable"] = mass_loss_det_lst

    # close file
    proc_data.close()

    if save_dict:
        save_file = save_dir + "/" + it_id + "_gc_mass_data.json"

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        with open(save_file, "w") as f:
            json.dump(mass_dict, f, indent=4)  # `indent=4` makes it more readable

    if return_dict:
        return mass_dict

In [72]:
# Save to a JSON file
evolve_mass_loss = 0.45

it_min = 0
it_max = 100

sim = "m12i"

sim_dir = "/Users/z5114326/Documents/simulations/"
save_dir = "../data/gc_mass_data/" + sim

In [76]:
for it in range(it_min, it_max + 1):
    get_mass_data(it, evolve_mass_loss, sim, sim_dir, save_dir, save_dict=True, return_dict=False)

# Lets plot

In [79]:
it = 0

it_id = iteration_name(it)

data_file = save_dir + "/" + it_id + "_gc_mass_data.json"

# open json file as dict
with open(data_file, "r") as file:
    mass_dict = json.load(file)

First we plot the mass of Ex-situ clusters as a function of accreted time

In [ ]:
mass_dict[it_id]["119998556"].keys()

dict_keys(['group_id', 'type_flag', 'time', 'form_time', 'log_mass', 'mass_loss', 'mass_loss_detectable'])

In [ ]:
plot_dict = {}

plot_dict[it_id] = {}

plot_dict[it_id]["0"] = {}
plot_dict[it_id]["1"] = {}
plot_dict[it_id]["2"] = {}
plot_dict[it_id]["3"] = {}
plot_dict[it_id]["4"] = {}

# set up dictionary
for type_flag in range(0, 5):
    plot_dict[it_id][str(type_flag)] = {}

    plot_dict[it_id][str(type_flag)]["time"] = []
    plot_dict[it_id][str(type_flag)]["form_time"] = []

    plot_dict[it_id][str(type_flag)]["mass"] = []
    plot_dict[it_id][str(type_flag)]["mass_loss"] = []
    plot_dict[it_id][str(type_flag)]["mass_loss_detectable"] = []

    if type_flag > 1:
        plot_dict[it_id][str(type_flag)]["acc_time"] = []

# fill type dict
for gc_id in mass_dict[it_id]:
    type_flag = mass_dict[it_id][gc_id]["type_flag"]

    time_lst = mass_dict[it_id][gc_id]["time"]
    form_time_lst = mass_dict[it_id][gc_id]["form_time"]

    log_mass_lst = mass_dict[it_id][gc_id]["log_mass"]

    mass_loss_lst = mass_dict[it_id][gc_id]["mass_loss"]
    mass_loss_detectable_lst = mass_dict[it_id][gc_id]["mass_loss_detectable"]

    if type_flag <= 1:
        for time, form_time, log_mass in zip(time_lst, form_time_lst, log_mass_lst):
            plot_dict[it_id][str(type_flag)]["time"].append(time)
            plot_dict[it_id][str(type_flag)]["form_time"].append(form_time)

            if log_mass is not None:
                mass = 10**log_mass
                plot_dict[it_id][str(type_flag)]["mass"].append(mass)
            else:
                plot_dict[it_id][str(type_flag)]["mass"].append(0)

    # plot_dict[it_id][str(type_flag)]["time"].append(mass_dict[it_id][gc_id]["time"])
    # plot_dict[it_id][str(type_flag)]["form_time"].append(mass_dict[it_id][gc_id]["form_time"])

    # # plot_dict[it_id][str(type_flag)]["mass"].append(10 ** (mass_dict[it_id][gc_id]["log_mass"]))
    # plot_dict[it_id][str(type_flag)]["mass_loss"].append(mass_dict[it_id][gc_id]["mass_loss"])
    # plot_dict[it_id][str(type_flag)]["mass_loss_detectable"].append(
    #     10 ** (mass_dict[it_id][gc_id]["mass_loss_detectable"])
    # )

    # if type_flag > 1:
    #     plot_dict[it_id][str(type_flag)]["acc_time"].append(mass_dict[it_id][gc_id]["acc_time"])


TypeError: unsupported operand type(s) for ** or pow(): 'int' and 'list'

In [103]:
for mass in mass_dict[it_id][gc_id]["log_mass"]:
    if mass is not None:
        print(mass)


4.029
3.762
3.735
3.709
3.67
3.611
3.575
3.476
3.335
2.948
